In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
import json
from sklearn.tree import export_graphviz
from sklearn.externals.six import StringIO  
from IPython.display import Image  
from sklearn.model_selection import KFold
import numpy as np
from sklearn.preprocessing import StandardScaler 
from sklearn import preprocessing

C:\Users\jim\Anaconda3\lib\site-packages\sklearn\externals\six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [2]:
with open('train.json') as f:
    data = json.load(f)
    
price = data['price']
latitude = data['latitude']
longitude = data['longitude']
time = data['created']

new_time = time
for keys in time:
    new_time[keys] = time[keys][11] + time[keys][12]

for keys in new_time:
    tmp = int(new_time[keys])
    new_time[keys] = tmp

In [3]:
def get_bounds(variable):
    values = list(variable.values())
    values.sort()
    length = len(values)
    inedx= int((length - 1)/4)
    q1 = (values[inedx] + values[inedx + 1])/2
    inedx = int((length - 1)*3/4)
    q3 = (values[inedx] + values[inedx + 1])/2
    iqr = q3 - q1
    lower_bound = q1 - 1.5*iqr            #100
    upper_bound = q3 + 1.5*iqr
    
    return lower_bound, upper_bound	

In [4]:
data=pd.DataFrame(data)

In [5]:

filter_missing_data = data[(data['features'].map(lambda d: len(d)) != 0) & (data['latitude'].map(lambda d: d) != 0) & (data['longitude'].map(lambda d: d) != 0)]
lower_bound,upper_bound = get_bounds(price)
filter_price_outlier_data = filter_missing_data[(filter_missing_data['price'] > lower_bound) & (filter_missing_data['price'] < upper_bound)]
filter_latitude_outlier_data = filter_price_outlier_data[(filter_price_outlier_data['latitude'].map(lambda d: d >= 40))]
filter_longitude_outlier_data = filter_latitude_outlier_data[(filter_latitude_outlier_data['longitude'].map(lambda d: d >= -79.8 and d <= -73.3))]

In [6]:
data = filter_longitude_outlier_data

In [7]:
data["num_photos"] = data["photos"].apply(len)
data["num_features"] = data["features"].apply(len)
data['street_address'] = data['street_address'].astype('category').cat.codes
data['building_id']=data['building_id'].astype('category').cat.codes
data['display_address'] = data['display_address'].astype('category').cat.codes
data['manager_id']=data['manager_id'].astype('category').cat.codes
data['interest_level']=data['interest_level'].astype('category').cat.codes

C:\Users\jim\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\jim\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\jim\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the doc

In [8]:
data = data.reset_index(drop=True)

In [9]:
feature_name = ['bathrooms',  'bedrooms','created', 'display_address', 'latitude', 'longitude', 'manager_id','price', 'num_photos', 'num_features']
x = data[feature_name]
y=data.interest_level

In [16]:
scores = 0
accuracy = 0
dtc = DecisionTreeClassifier(criterion='gini',max_depth = 6)
cv5 = KFold(n_splits=5)
cv5.get_n_splits(X=x)
print(cv5)
for train_index, test_index in cv5.split(range(data.shape[0])):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = x.iloc[train_index,:], x.iloc[test_index,:]
    y_train, y_test = y[train_index], y[test_index]
    dtc.fit(X_train,y_train)
    y_pred = dtc.predict_proba(X_test)
    current_accuracy = metrics.accuracy_score(y_test, dtc.predict(X_test))
    current_logloss = metrics.log_loss(y_test,y_pred)
    print("Accuracy:", current_accuracy)
    print("Log loss:", current_logloss)
    scores = scores + current_logloss
    accuracy = accuracy + current_accuracy

KFold(n_splits=5, random_state=None, shuffle=False)
TRAIN: [ 8642  8643  8644 ... 43207 43208 43209] TEST: [   0    1    2 ... 8639 8640 8641]
Accuracy: 0.6962508678546633
Log loss: 0.7185483970916013
TRAIN: [    0     1     2 ... 43207 43208 43209] TEST: [ 8642  8643  8644 ... 17281 17282 17283]
Accuracy: 0.6930108771117797
Log loss: 0.7072139588263745
TRAIN: [    0     1     2 ... 43207 43208 43209] TEST: [17284 17285 17286 ... 25923 25924 25925]
Accuracy: 0.6861837537607035
Log loss: 0.746630551014909
TRAIN: [    0     1     2 ... 43207 43208 43209] TEST: [25926 25927 25928 ... 34565 34566 34567]
Accuracy: 0.6869937514464244
Log loss: 0.7314887298893381
TRAIN: [    0     1     2 ... 34565 34566 34567] TEST: [34568 34569 34570 ... 43207 43208 43209]
Accuracy: 0.6913908817403379
Log loss: 0.7236408891532103


In [17]:
avg_scores = scores / 5

In [18]:
avg_scores

0.7255045051950866

In [19]:
avg_accuracy = accuracy / 5

In [20]:
avg_accuracy

0.6907660263827817

In [39]:
feature_name = ['bathrooms',  'bedrooms','created', 'display_address', 'latitude', 'longitude', 'manager_id','price', 'num_photos', 'num_features']
x = data[feature_name]
y=data.interest_level

In [22]:
x

,bathrooms,bedrooms,created,display_address,latitude,longitude,manager_id,price,num_photos,num_features
0,1.0,1,5,732,40.7108,-73.9539,2038,2400,12,7
1,1.0,2,5,4770,40.7513,-73.9722,1896,3800,6,6
2,1.0,2,15,4796,40.7575,-73.9625,2561,3495,6,6
3,1.0,0,3,4736,40.7439,-73.9743,592,2795,4,4
4,2.0,3,6,4663,40.7302,-73.9826,2009,6000,7,5
...,...,...,...,...,...,...,...,...,...,...
43205,1.0,3,3,6370,40.8433,-73.9396,2100,2800,5,3
43206,1.0,2,2,3614,40.8198,-73.9578,1816,2395,5,7
43207,1.0,1,5,1177,40.5765,-73.9554,2038,1850,3,11
43208,1.0,2,2,7128,40.7448,-74.0017,2494,4195,5,5


In [26]:
scores = 0
accuracy = 0
log = LogisticRegression(penalty = 'l2', solver = 'newton-cg', multi_class ='multinomial', max_iter = 10000)
cv5 = KFold(n_splits=5)
cv5.get_n_splits(X=x)
print(cv5)
for train_index, test_index in cv5.split(range(data.shape[0])):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = x.iloc[train_index,:], x.iloc[test_index,:]
    y_train, y_test = y[train_index], y[test_index]
    log.fit(X_train,y_train)
    y_pred = log.predict_proba(X_test)
    current_accuracy = metrics.accuracy_score(y_test, log.predict(X_test))
    current_logloss = metrics.log_loss(y_test,y_pred)
    print("Accuracy:", current_accuracy)
    print("Log loss:", current_logloss)
    scores = scores + current_logloss
    accuracy = accuracy + current_accuracy

KFold(n_splits=5, random_state=None, shuffle=False)
TRAIN: [ 8642  8643  8644 ... 43207 43208 43209] TEST: [   0    1    2 ... 8639 8640 8641]


C:\Users\jim\Anaconda3\lib\site-packages\scipy\optimize\linesearch.py:466: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
C:\Users\jim\Anaconda3\lib\site-packages\scipy\optimize\linesearch.py:314: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
C:\Users\jim\Anaconda3\lib\site-packages\scipy\optimize\linesearch.py:426: LineSearchWarning: Rounding errors prevent the line search from converging
  warn(msg, LineSearchWarning)
C:\Users\jim\Anaconda3\lib\site-packages\sklearn\utils\optimize.py:195: UserWarning: Line Search failed
  warnings.warn('Line Search failed')


Accuracy: 0.6874566072668363
Log loss: 0.7170667431638184
TRAIN: [    0     1     2 ... 43207 43208 43209] TEST: [ 8642  8643  8644 ... 17281 17282 17283]


C:\Users\jim\Anaconda3\lib\site-packages\scipy\optimize\linesearch.py:466: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
C:\Users\jim\Anaconda3\lib\site-packages\scipy\optimize\linesearch.py:314: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
C:\Users\jim\Anaconda3\lib\site-packages\sklearn\utils\optimize.py:195: UserWarning: Line Search failed
  warnings.warn('Line Search failed')


Accuracy: 0.6843323304790557
Log loss: 0.7209154282085615
TRAIN: [    0     1     2 ... 43207 43208 43209] TEST: [17284 17285 17286 ... 25923 25924 25925]


...

C:\Users\jim\Anaconda3\lib\site-packages\scipy\optimize\linesearch.py:466: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
C:\Users\jim\Anaconda3\lib\site-packages\scipy\optimize\linesearch.py:314: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
C:\Users\jim\Anaconda3\lib\site-packages\sklearn\utils\optimize.py:195: UserWarning: Line Search failed
  warnings.warn('Line Search failed')


Accuracy: 0.6742652163850961
Log loss: 0.7384857092990879
TRAIN: [    0     1     2 ... 34565 34566 34567] TEST: [34568 34569 34570 ... 43207 43208 43209]


C:\Users\jim\Anaconda3\lib\site-packages\scipy\optimize\linesearch.py:466: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
C:\Users\jim\Anaconda3\lib\site-packages\scipy\optimize\linesearch.py:314: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)


Accuracy: 0.6824809071974081
Log loss: 0.7262498115086246


C:\Users\jim\Anaconda3\lib\site-packages\scipy\optimize\linesearch.py:426: LineSearchWarning: Rounding errors prevent the line search from converging
  warn(msg, LineSearchWarning)
C:\Users\jim\Anaconda3\lib\site-packages\sklearn\utils\optimize.py:195: UserWarning: Line Search failed
  warnings.warn('Line Search failed')


In [30]:
avg_scores = scores / 5

[0.7231100805470594,
 0.7311302379172795,
 0.7380343932243466,
 0.7467555292743658,
 0.7328185254062372]

In [27]:
avg_scores

0.725463038880131

In [28]:
avg_accuracy = accuracy / 5

In [29]:
avg_accuracy

0.6810229113631104

In [14]:
feature_name = ['price','display_address']
x = data[feature_name]
y=data.interest_level
cv5 = KFold(n_splits=5)
cv5.get_n_splits(X=x)
print(cv5)

KFold(n_splits=5, random_state=None, shuffle=False)


In [ ]:
scores = 0
accuracy = 0
svm = SVC(kernel='linear',probability=True, C= 0.03125)
cv5 = KFold(n_splits=5)
cv5.get_n_splits(X=x)
print(cv5)
for train_index, test_index in cv5.split(range(data1.shape[0])):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = x.iloc[train_index,:], x.iloc[test_index,:]
    y_train, y_test = y[train_index], y[test_index]
    svm.fit(X_train,y_train)
    y_pred = svm.predict_proba(X_test)
    current_accuracy = metrics.accuracy_score(y_test, log.predict(X_test))
    current_logloss = metrics.log_loss(y_test,y_pred)
    print("Accuracy:", current_accuracy)
    print("Log loss:", current_logloss)
    scores = scores + current_logloss
    accuracy = accuracy + current_accuracy

KFold(n_splits=5, random_state=None, shuffle=False)
TRAIN: [ 600  601  602 ... 2994 2995 2996] TEST: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 22

In [15]:
y.shape

(2997,)

In [1]:
avg_scores = scores / 5
avg_accuracy = accuracy / 5

NameError: name 'scores' is not defined

In [ ]:
avg_scores

In [ ]:
avg_accuracy